In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import matplotlib.pyplot as plt
import IPython.display as idis
import seaborn as sns
import copy
import scipy.stats as stat
import functools 
%matplotlib inline
pd.options.display.max_rows = 999
pd.options.display.max_columns = 300

In [2]:
#load data

Mimic_data = '/home/student/huy/mnt/mrAnKhuongData/storage/data/curated/all_hourly_data.h5'
vitals_data = pd.read_hdf(Mimic_data, 'vitals_labs')
interventions = pd.read_hdf(Mimic_data, 'interventions')
patients = pd.read_hdf(Mimic_data, 'patients')


In [3]:
#get shape of data

print(vitals_data.shape)
print(interventions.shape)
print(patients.shape)

(3768901, 312)
(3768901, 14)
(38604, 17)


In [4]:
patients.head(40)

,,,gender,ethnicity,age,insurance,admittime,dischtime,deathtime,intime,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,max_hours
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,
3,145834,211552,M,WHITE,76.5268,Medicare,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646,EMERGENCY,MICU,0,0,0,1,145
4,185777,294638,F,WHITE,47.8450,Private,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785,EMERGENCY,MICU,0,0,0,1,40
6,107064,228232,F,WHITE,65.9407,Medicare,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729,ELECTIVE,SICU,0,0,0,1,88
9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.7902,Medicaid,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,2149-11-09 13:07:02,2149-11-14 20:52:14,5.3231,EMERGENCY,MICU,1,1,1,1,127
11,194540,229441,F,WHITE,50.1483,Private,2178-04-16 06:18:00,2178-05-11 19:00:00,NaT,2178-04-16 06:19:32,2178-04-17 20:21:05,1.5844,EMERGENCY,SICU,0,0,0,1,38
12,112213,232669,M,WHITE,72.3724,Medicare,2104-08-07 10:15:00,2104-08-20 02:57:00,2104-08-20 02:57:00,2104-08-08 02:08:17,2104-08-15 17:22:25,7.6348,ELECTIVE,SICU,0,1,1,1,183
13,143045,263738,F,WHITE,39.8661,Medicaid,2167-01-08 18:43:00,2167-01-15 15:15:00,NaT,2167-01-08 18:44:25,2167-01-12 10:43:31,3.6660,EMERGENCY,CCU,0,0,0,1,87
17,194023,277042,F,WHITE,47.4543,Private,2134-12-27 07:15:00,2134-12-31 16:05:00,NaT,2134-12-27 16:21:48,2134-12-29 18:04:03,2.0710,ELECTIVE,CSRU,0,0,0,1,49
18,188822,298129,M,WHITE,50.8416,Private,2167-10-02 11:18:00,2167-10-04 16:15:00,NaT,2167-10-02 11:20:39,2167-10-03 18:16:05,1.2885,EMERGENCY,CCU,0,0,0,1,30


In [5]:
# list_nat_admittime = [patients.index[index][0] for index in range(0, patients.shape[0]) if type(patients.iloc[index,patients.columns.get_loc('intime')]) == pd._libs.tslibs.nattype.NaTType]

# list_nat_admittime

In [6]:
# patients = patients.drop(list_nat_admittime)
# patients.head(30)

In [7]:
# Abnormality behavior : admit time (hospital) after in time (icu)
# If admit time (hospital) after in time (icu) (see sorted list gap time in explore_data_mimic file), we will change
# the admit time to icu time

# abnormal_amittime_patients_index = list(patients.loc[patients['admittime'] > patients['intime']].index.get_level_values('subject_id'))

# for subject_id in abnormal_amittime_patients_index:
#     patients.loc[subject_id,'admittime'] = patients.loc[subject_id,'intime']
    
# patients.loc[patients['admittime'] > patients['intime']]

abnormal_amittime_patients_index = functools.reduce(lambda x, y: x + [y] if patients.iloc[y,patients.columns.get_loc('admittime')] > patients.iloc[y,patients.columns.get_loc('intime')] else x +[], [i for i in range(0,patients.shape[0])] ,[])

abnormal_amittime_patients_index


for index in abnormal_amittime_patients_index:
    patients.iloc[index,patients.columns.get_loc('admittime')] = patients.iloc[index,patients.columns.get_loc('intime')]

patients.loc[patients['admittime'] > patients['intime']]

,,,gender,ethnicity,age,insurance,admittime,dischtime,deathtime,intime,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,max_hours
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,


In [8]:
# patients.iloc[[2,4],patients.columns.get_loc('deathtime')] = patients.iloc[0,patients.columns.get_loc('admittime')]
# sum(pd.isna(patients.iloc[:,patients.columns.get_loc('admittime')]))
# type(patients.iloc[2,patients.columns.get_loc('deathtime')])==pd._libs.tslibs.nattype.NaTType
# patients = patients.drop(list_NaT_deathtime)
# patients.iloc[6,patients.columns.get_loc('deathtime')] = patients.iloc[6,patients.columns.get_loc('intime')]

patients.head(40)

,,,gender,ethnicity,age,insurance,admittime,dischtime,deathtime,intime,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,max_hours
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,
3,145834,211552,M,WHITE,76.5268,Medicare,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646,EMERGENCY,MICU,0,0,0,1,145
4,185777,294638,F,WHITE,47.8450,Private,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785,EMERGENCY,MICU,0,0,0,1,40
6,107064,228232,F,WHITE,65.9407,Medicare,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729,ELECTIVE,SICU,0,0,0,1,88
9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.7902,Medicaid,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,2149-11-09 13:07:02,2149-11-14 20:52:14,5.3231,EMERGENCY,MICU,1,1,1,1,127
11,194540,229441,F,WHITE,50.1483,Private,2178-04-16 06:18:00,2178-05-11 19:00:00,NaT,2178-04-16 06:19:32,2178-04-17 20:21:05,1.5844,EMERGENCY,SICU,0,0,0,1,38
12,112213,232669,M,WHITE,72.3724,Medicare,2104-08-07 10:15:00,2104-08-20 02:57:00,2104-08-20 02:57:00,2104-08-08 02:08:17,2104-08-15 17:22:25,7.6348,ELECTIVE,SICU,0,1,1,1,183
13,143045,263738,F,WHITE,39.8661,Medicaid,2167-01-08 18:43:00,2167-01-15 15:15:00,NaT,2167-01-08 18:44:25,2167-01-12 10:43:31,3.6660,EMERGENCY,CCU,0,0,0,1,87
17,194023,277042,F,WHITE,47.4543,Private,2134-12-27 07:15:00,2134-12-31 16:05:00,NaT,2134-12-27 16:21:48,2134-12-29 18:04:03,2.0710,ELECTIVE,CSRU,0,0,0,1,49
18,188822,298129,M,WHITE,50.8416,Private,2167-10-02 11:18:00,2167-10-04 16:15:00,NaT,2167-10-02 11:20:39,2167-10-03 18:16:05,1.2885,EMERGENCY,CCU,0,0,0,1,30


In [9]:
# Abnormal behavior: Some patients have admittime > dischtime

patients.loc[patients['admittime'] > patients['dischtime']]

#delete patients with abnormal behavior above

list_abnormal_admit_disch_gap = list(patients.loc[patients['admittime'] > patients['dischtime']].index.get_level_values('subject_id'))

patients = patients.drop(list_abnormal_admit_disch_gap)

#delete vitals lab of patients with abnormal behavior above
vitals_data = vitals_data.drop(list_abnormal_admit_disch_gap)

# check again in two table patients and vitals_data
print(patients.loc[patients['admittime'] > patients['dischtime']])
print(vitals_data.loc[list_abnormal_admit_disch_gap, :])


Empty DataFrame
Columns: [gender, ethnicity, age, insurance, admittime, dischtime, deathtime, intime, outtime, los_icu, admission_type, first_careunit, mort_icu, mort_hosp, hospital_expire_flag, hospstay_seq, max_hours]
Index: []
Empty DataFrame
Columns: [(alanine aminotransferase, count), (alanine aminotransferase, mean), (alanine aminotransferase, std), (albumin, count), (albumin, mean), (albumin, std), (albumin ascites, count), (albumin ascites, mean), (albumin ascites, std), (albumin pleural, count), (albumin pleural, mean), (albumin pleural, std), (albumin urine, count), (albumin urine, mean), (albumin urine, std), (alkaline phosphate, count), (alkaline phosphate, mean), (alkaline phosphate, std), (anion gap, count), (anion gap, mean), (anion gap, std), (asparate aminotransferase, count), (asparate aminotransferase, mean), (asparate aminotransferase, std), (basophils, count), (basophils, mean), (basophils, std), (bicarbonate, count), (bicarbonate, mean), (bicarbonate, std), (bilir

In [10]:
patients.head(10)

,,,gender,ethnicity,age,insurance,admittime,dischtime,deathtime,intime,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,max_hours
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,
3,145834,211552,M,WHITE,76.5268,Medicare,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646,EMERGENCY,MICU,0,0,0,1,145
4,185777,294638,F,WHITE,47.8450,Private,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785,EMERGENCY,MICU,0,0,0,1,40
6,107064,228232,F,WHITE,65.9407,Medicare,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729,ELECTIVE,SICU,0,0,0,1,88
9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.7902,Medicaid,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,2149-11-09 13:07:02,2149-11-14 20:52:14,5.3231,EMERGENCY,MICU,1,1,1,1,127
11,194540,229441,F,WHITE,50.1483,Private,2178-04-16 06:18:00,2178-05-11 19:00:00,NaT,2178-04-16 06:19:32,2178-04-17 20:21:05,1.5844,EMERGENCY,SICU,0,0,0,1,38
12,112213,232669,M,WHITE,72.3724,Medicare,2104-08-07 10:15:00,2104-08-20 02:57:00,2104-08-20 02:57:00,2104-08-08 02:08:17,2104-08-15 17:22:25,7.6348,ELECTIVE,SICU,0,1,1,1,183
13,143045,263738,F,WHITE,39.8661,Medicaid,2167-01-08 18:43:00,2167-01-15 15:15:00,NaT,2167-01-08 18:44:25,2167-01-12 10:43:31,3.6660,EMERGENCY,CCU,0,0,0,1,87
17,194023,277042,F,WHITE,47.4543,Private,2134-12-27 07:15:00,2134-12-31 16:05:00,NaT,2134-12-27 16:21:48,2134-12-29 18:04:03,2.0710,ELECTIVE,CSRU,0,0,0,1,49
18,188822,298129,M,WHITE,50.8416,Private,2167-10-02 11:18:00,2167-10-04 16:15:00,NaT,2167-10-02 11:20:39,2167-10-03 18:16:05,1.2885,EMERGENCY,CCU,0,0,0,1,30


In [11]:
# add column total hospital LOS (dischtime -admittime)

total_los_series = pd.Series(list(map(lambda x: round(x.total_seconds()/(24*60*60),4),(patients.iloc[:,patients.columns.get_loc('dischtime')] - patients.iloc[:,patients.columns.get_loc('admittime')]))))
patients['total_los'] = total_los_series.values

In [12]:
# No nan value in total_los column
sum(pd.isna(patients['total_los']))

0

In [13]:
patients.head(36)

,,,gender,ethnicity,age,insurance,admittime,dischtime,deathtime,intime,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,max_hours,total_los
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,
3,145834,211552,M,WHITE,76.5268,Medicare,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646,EMERGENCY,MICU,0,0,0,1,145,10.7847
4,185777,294638,F,WHITE,47.8450,Private,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785,EMERGENCY,MICU,0,0,0,1,40,7.7590
6,107064,228232,F,WHITE,65.9407,Medicare,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729,ELECTIVE,SICU,0,0,0,1,88,16.3646
9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.7902,Medicaid,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,2149-11-09 13:07:02,2149-11-14 20:52:14,5.3231,EMERGENCY,MICU,1,1,1,1,127,4.8812
11,194540,229441,F,WHITE,50.1483,Private,2178-04-16 06:18:00,2178-05-11 19:00:00,NaT,2178-04-16 06:19:32,2178-04-17 20:21:05,1.5844,EMERGENCY,SICU,0,0,0,1,38,25.5292
12,112213,232669,M,WHITE,72.3724,Medicare,2104-08-07 10:15:00,2104-08-20 02:57:00,2104-08-20 02:57:00,2104-08-08 02:08:17,2104-08-15 17:22:25,7.6348,ELECTIVE,SICU,0,1,1,1,183,12.6958
13,143045,263738,F,WHITE,39.8661,Medicaid,2167-01-08 18:43:00,2167-01-15 15:15:00,NaT,2167-01-08 18:44:25,2167-01-12 10:43:31,3.6660,EMERGENCY,CCU,0,0,0,1,87,6.8556
17,194023,277042,F,WHITE,47.4543,Private,2134-12-27 07:15:00,2134-12-31 16:05:00,NaT,2134-12-27 16:21:48,2134-12-29 18:04:03,2.0710,ELECTIVE,CSRU,0,0,0,1,49,4.3681
18,188822,298129,M,WHITE,50.8416,Private,2167-10-02 11:18:00,2167-10-04 16:15:00,NaT,2167-10-02 11:20:39,2167-10-03 18:16:05,1.2885,EMERGENCY,CCU,0,0,0,1,30,2.2062


In [14]:
# Many patients hav Unknown admitime (nat), leading to nan total_los. We can not check NaT, so we use nan in total_los
# column to delete these patients

# list_NaT_admitime = patients.loc[(pd.isna(list(patients.total_los)))].index.get_level_values('subject_id')
# patients = patients.drop(list_NaT_admitime)
# vitals_data = vitals_data.drop(list_NaT_admitime)


In [15]:
patients.head(25)

,,,gender,ethnicity,age,insurance,admittime,dischtime,deathtime,intime,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,max_hours,total_los
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,
3,145834,211552,M,WHITE,76.5268,Medicare,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646,EMERGENCY,MICU,0,0,0,1,145,10.7847
4,185777,294638,F,WHITE,47.8450,Private,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785,EMERGENCY,MICU,0,0,0,1,40,7.7590
6,107064,228232,F,WHITE,65.9407,Medicare,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729,ELECTIVE,SICU,0,0,0,1,88,16.3646
9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.7902,Medicaid,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,2149-11-09 13:07:02,2149-11-14 20:52:14,5.3231,EMERGENCY,MICU,1,1,1,1,127,4.8812
11,194540,229441,F,WHITE,50.1483,Private,2178-04-16 06:18:00,2178-05-11 19:00:00,NaT,2178-04-16 06:19:32,2178-04-17 20:21:05,1.5844,EMERGENCY,SICU,0,0,0,1,38,25.5292
12,112213,232669,M,WHITE,72.3724,Medicare,2104-08-07 10:15:00,2104-08-20 02:57:00,2104-08-20 02:57:00,2104-08-08 02:08:17,2104-08-15 17:22:25,7.6348,ELECTIVE,SICU,0,1,1,1,183,12.6958
13,143045,263738,F,WHITE,39.8661,Medicaid,2167-01-08 18:43:00,2167-01-15 15:15:00,NaT,2167-01-08 18:44:25,2167-01-12 10:43:31,3.6660,EMERGENCY,CCU,0,0,0,1,87,6.8556
17,194023,277042,F,WHITE,47.4543,Private,2134-12-27 07:15:00,2134-12-31 16:05:00,NaT,2134-12-27 16:21:48,2134-12-29 18:04:03,2.0710,ELECTIVE,CSRU,0,0,0,1,49,4.3681
18,188822,298129,M,WHITE,50.8416,Private,2167-10-02 11:18:00,2167-10-04 16:15:00,NaT,2167-10-02 11:20:39,2167-10-03 18:16:05,1.2885,EMERGENCY,CCU,0,0,0,1,30,2.2062


In [16]:
sum(pd.isna(patients['los_icu']))

0

In [17]:
# abnormal behavior: many hospital LOS < ICU LOS

gap_los_icu_hospital = list(patients['total_los'] - patients['los_icu'])


# print("Number patients with total hospital LOS < icu los: ", sum(np.array(gap_los_icu_hospital)<0))
print(len(sorted([a for a in gap_los_icu_hospital if a<=-0.5])))



373


In [18]:
print(patients.shape)
print(vitals_data.shape)

(38541, 18)
(3768237, 312)


In [19]:
#delete patients with total LOS < patients LOS ( to keep as much data as possible, we delete with below condition)

list_los_total_smaller_icu = patients.loc[(patients['total_los'] - patients['los_icu']) <= -0.5].index.get_level_values('subject_id')

patients = patients.drop(list_los_total_smaller_icu)
vitals_data = vitals_data.drop(list_los_total_smaller_icu)

print(patients.shape)
print(vitals_data.shape)

(38168, 18)
(3708663, 312)


In [20]:
patients.head(30)

,,,gender,ethnicity,age,insurance,admittime,dischtime,deathtime,intime,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,max_hours,total_los
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,
3,145834,211552,M,WHITE,76.5268,Medicare,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646,EMERGENCY,MICU,0,0,0,1,145,10.7847
4,185777,294638,F,WHITE,47.8450,Private,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785,EMERGENCY,MICU,0,0,0,1,40,7.7590
6,107064,228232,F,WHITE,65.9407,Medicare,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729,ELECTIVE,SICU,0,0,0,1,88,16.3646
9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.7902,Medicaid,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,2149-11-09 13:07:02,2149-11-14 20:52:14,5.3231,EMERGENCY,MICU,1,1,1,1,127,4.8812
11,194540,229441,F,WHITE,50.1483,Private,2178-04-16 06:18:00,2178-05-11 19:00:00,NaT,2178-04-16 06:19:32,2178-04-17 20:21:05,1.5844,EMERGENCY,SICU,0,0,0,1,38,25.5292
12,112213,232669,M,WHITE,72.3724,Medicare,2104-08-07 10:15:00,2104-08-20 02:57:00,2104-08-20 02:57:00,2104-08-08 02:08:17,2104-08-15 17:22:25,7.6348,ELECTIVE,SICU,0,1,1,1,183,12.6958
13,143045,263738,F,WHITE,39.8661,Medicaid,2167-01-08 18:43:00,2167-01-15 15:15:00,NaT,2167-01-08 18:44:25,2167-01-12 10:43:31,3.6660,EMERGENCY,CCU,0,0,0,1,87,6.8556
17,194023,277042,F,WHITE,47.4543,Private,2134-12-27 07:15:00,2134-12-31 16:05:00,NaT,2134-12-27 16:21:48,2134-12-29 18:04:03,2.0710,ELECTIVE,CSRU,0,0,0,1,49,4.3681
18,188822,298129,M,WHITE,50.8416,Private,2167-10-02 11:18:00,2167-10-04 16:15:00,NaT,2167-10-02 11:20:39,2167-10-03 18:16:05,1.2885,EMERGENCY,CCU,0,0,0,1,30,2.2062


In [21]:
# if los_icu > total_los => total_los := los icu

abnormal_icu_total_gap_index = functools.reduce(lambda x, y: x + [y] if patients.iloc[y,patients.columns.get_loc('total_los')] < patients.iloc[y,patients.columns.get_loc('los_icu')] else x +[], [i for i in range(0,patients.shape[0])] ,[])

abnormal_icu_total_gap_index


for index in abnormal_icu_total_gap_index:
    patients.iloc[index,patients.columns.get_loc('total_los')] = patients.iloc[index,patients.columns.get_loc('los_icu')]


patients.loc[patients['total_los'] < patients['los_icu']]


,,,gender,ethnicity,age,insurance,admittime,dischtime,deathtime,intime,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,max_hours,total_los
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,


In [22]:
patients.shape

(38168, 18)

In [23]:
# Filtering patients (subject_id) in vitals_data table based on patient in patients table 
# Get index in patients table
icustay_id_index = set(patients.index.get_level_values('icustay_id'))

#filtering vitals lab
vitals_data_filtered = vitals_data[vitals_data.index.get_level_values('icustay_id').isin(icustay_id_index)]

In [24]:
print(patients.shape)
print(vitals_data_filtered.shape)

(38168, 18)
(3708663, 312)


In [25]:
# Filtering vitals lab collected in the first 24h after first ICU admission and patients with icu_los at least window_time
# + gaptime (gap_time should be 1 to keep data)
# Gap time to avoid data leakage and make predicting model really useful in practice
# Also, Fil

WINDOW_SIZE = 24  
GAP_TIME = 1

Ys = patients[patients.max_hours > WINDOW_SIZE + GAP_TIME][['mort_hosp', 'mort_icu', 'los_icu']]

patients = patients[patients.index.get_level_values('icustay_id').isin(set(Ys.index.get_level_values('icustay_id')))]

vitals_data_filtered = vitals_data_filtered[(vitals_data_filtered.index.get_level_values('icustay_id').isin(set(Ys.index.get_level_values('icustay_id')))) & (vitals_data_filtered.index.get_level_values('hours_in') < WINDOW_SIZE)]

In [26]:
# Matching patients table and vitals table

# matching number patients
print(patients.shape)
print(vitals_data_filtered.shape)

# matching all patients and its order(subject_id)
a= set(list(vitals_data_filtered.index.get_level_values('subject_id')))
assert  a == set(list(vitals_data_filtered.index.get_level_values('subject_id')))
a == set(list(patients.index.get_level_values('subject_id')))

(30495, 18)
(731880, 312)


True

In [27]:
patients[(patients['total_los'] > 30) ].loc[:,['age','total_los']].mean()

age          62.584285
total_los    45.920751
dtype: float64

In [28]:
# Check again icu_los and total_los to ensure no abnormal behavior

sorted(list(patients['total_los']))
# sorted(list(patients['los_icu']))

[1.0859,
 1.0862,
 1.0864,
 1.0877,
 1.0893,
 1.092,
 1.0933,
 1.095,
 1.0951,
 1.0959,
 1.0971,
 1.0993,
 1.0995,
 1.1,
 1.1002,
 1.1004,
 1.1005,
 1.1028,
 1.103,
 1.1046,
 1.1069,
 1.1076,
 1.1077,
 1.1082,
 1.1083,
 1.1094,
 1.1102,
 1.1104,
 1.1105,
 1.1105,
 1.1108,
 1.1115,
 1.1118,
 1.1118,
 1.1126,
 1.114,
 1.1141,
 1.1153,
 1.1162,
 1.1165,
 1.1167,
 1.117,
 1.1182,
 1.119,
 1.1194,
 1.1203,
 1.1204,
 1.1206,
 1.1217,
 1.1217,
 1.1223,
 1.1244,
 1.1244,
 1.125,
 1.1257,
 1.1262,
 1.1264,
 1.1268,
 1.1305,
 1.1313,
 1.1313,
 1.1321,
 1.1327,
 1.1327,
 1.1347,
 1.1347,
 1.1349,
 1.1357,
 1.1357,
 1.137,
 1.1379,
 1.1383,
 1.1383,
 1.1383,
 1.1389,
 1.1389,
 1.139,
 1.1396,
 1.1399,
 1.1405,
 1.141,
 1.141,
 1.1412,
 1.1424,
 1.1431,
 1.1441,
 1.1444,
 1.1445,
 1.1449,
 1.145,
 1.1451,
 1.1451,
 1.1455,
 1.1457,
 1.1458,
 1.1458,
 1.1458,
 1.1459,
 1.1472,
 1.1481,
 1.1484,
 1.1488,
 1.15,
 1.15,
 1.15,
 1.15,
 1.151,
 1.151,
 1.1512,
 1.1514,
 1.1525,
 1.1528,
 1.1534,
 1.1542,

In [29]:
# vitals_data_filtered.head(200)

## Feature reorganization 

<ol>
<li>Taking first time measurements of each vital labs </li>
<li>Computing the "trajectory of each vitals lab if they are measured multiple times (last__time_masure - first_time_measure)</li>
<li>Including static patients' information</li>
<li>(Optional) Including intervention features</li>
</ol>

In [42]:
# Get list vital_lab_name
vital_lab_name = list(vitals_data.columns.levels[0])
vital_lab_name = [c.replace(' ','_') for c in vital_lab_name]

# double vital_lab_list (ex: albumin for first time taking measure and albumin_last for last time taking measure)
vital_name_first_last = []
for i in vital_lab_name:
    vital_name_first_last.append(i)
    vital_name_first_last.append(str(i + '_last'))


# Create new dataframe with intervention name as column
patient_data = pd.DataFrame(columns = vital_name_first_last)

# Get subject id of all patient
patient_id = patients.index.get_level_values('subject_id')

# Add first observed measure of each vitals lab to patient_data
idx = pd.IndexSlice

for p_id in patient_id:
    a = vitals_data_filtered.loc[idx[p_id,:,:,:], idx[:,'mean']]
    vital_each_patient = []
    for i in range(0,104):
        if a.iloc[:,i].first_valid_index() == None:
            vital_each_patient.append(np.nan)
        else:
            index = a.iloc[:,i].first_valid_index()[3]
            vital_each_patient.append(a.iloc[index, i])
#             if a.iloc[index+1:,i].last_valid_index() != None:
#                 lst_last.append(a.iloc[a.iloc[index+1:,i].last_valid_index()[3], i]

        if a.iloc[:,i].last_valid_index() == None:
            vital_each_patient.append(np.nan)
    #         lst_last.append(np.nan)
        else:
            index = a.iloc[:,i].last_valid_index()[3]
            vital_each_patient.append(a.iloc[index, i])
#     print(len(vital_name_first_last))      
#     print(vital_name_first_last)
#     print(vital_each_patient)
#     break
    patient_data.loc[p_id] = vital_each_patient


# patient_data.head(50)


In [43]:
patient_data.head()

,alanine_aminotransferase,alanine_aminotransferase_last,albumin,albumin_last,albumin_ascites,albumin_ascites_last,albumin_pleural,albumin_pleural_last,albumin_urine,albumin_urine_last,alkaline_phosphate,alkaline_phosphate_last,anion_gap,anion_gap_last,asparate_aminotransferase,asparate_aminotransferase_last,basophils,basophils_last,bicarbonate,bicarbonate_last,bilirubin,bilirubin_last,blood_urea_nitrogen,blood_urea_nitrogen_last,calcium,calcium_last,calcium_ionized,calcium_ionized_last,calcium_urine,calcium_urine_last,cardiac_index,cardiac_index_last,cardiac_output_fick,cardiac_output_fick_last,cardiac_output_thermodilution,cardiac_output_thermodilution_last,central_venous_pressure,central_venous_pressure_last,chloride,chloride_last,chloride_urine,chloride_urine_last,cholesterol,cholesterol_last,cholesterol_hdl,cholesterol_hdl_last,cholesterol_ldl,cholesterol_ldl_last,co2,co2_last,"co2_(etco2,_pco2,_etc.)","co2_(etco2,_pco2,_etc.)_last",creatinine,creatinine_last,creatinine_ascites,creatinine_ascites_last,creatinine_body_fluid,creatinine_body_fluid_last,creatinine_pleural,creatinine_pleural_last,creatinine_urine,creatinine_urine_last,diastolic_blood_pressure,diastolic_blood_pressure_last,eosinophils,eosinophils_last,fibrinogen,fibrinogen_last,fraction_inspired_oxygen,fraction_inspired_oxygen_last,fraction_inspired_oxygen_set,fraction_inspired_oxygen_set_last,glascow_coma_scale_total,glascow_coma_scale_total_last,glucose,glucose_last,heart_rate,heart_rate_last,height,height_last,hematocrit,hematocrit_last,hemoglobin,hemoglobin_last,lactate,lactate_last,lactate_dehydrogenase,lactate_dehydrogenase_last,lactate_dehydrogenase_pleural,lactate_dehydrogenase_pleural_last,lactic_acid,lactic_acid_last,lymphocytes,lymphocytes_last,lymphocytes_ascites,lymphocytes_ascites_last,lymphocytes_atypical,lymphocytes_atypical_last,lymphocytes_atypical_csl,lymphocytes_atypical_csl_last,lymphocytes_body_fluid,lymphocytes_body_fluid_last,lymphocytes_percent,lymphocytes_percent_last,lymphocytes_pleural,lymphocytes_pleural_last,magnesium,magnesium_last,mean_blood_pressure,mean_blood_pressure_last,mean_corpuscular_hemoglobin,mean_corpuscular_hemoglobin_last,mean_corpuscular_hemoglobin_concentration,mean_corpuscular_hemoglobin_concentration_last,mean_corpuscular_volume,mean_corpuscular_volume_last,monocytes,monocytes_last,monocytes_csl,monocytes_csl_last,neutrophils,neutrophils_last,oxygen_saturation,oxygen_saturation_last,partial_pressure_of_carbon_dioxide,partial_pressure_of_carbon_dioxide_last,partial_pressure_of_oxygen,partial_pressure_of_oxygen_last,partial_thromboplastin_time,partial_thromboplastin_time_last,peak_inspiratory_pressure,peak_inspiratory_pressure_last,ph,ph_last,ph_urine,ph_urine_last,phosphate,phosphate_last,phosphorous,phosphorous_last,plateau_pressure,plateau_pressure_last,platelets,platelets_last,positive_end-expiratory_pressure,positive_end-expiratory_pressure_last,positive_end-expiratory_pressure_set,positive_end-expiratory_pressure_set_last,post_void_residual,post_void_residual_last,potassium,potassium_last,potassium_serum,potassium_serum_last,prothrombin_time_inr,prothrombin_time_inr_last,prothrombin_time_pt,prothrombin_time_pt_last,pulmonary_artery_pressure_mean,pulmonary_artery_pressure_mean_last,pulmonary_artery_pressure_systolic,pulmonary_artery_pressure_systolic_last,pulmonary_capillary_wedge_pressure,pulmonary_capillary_wedge_pressure_last,red_blood_cell_count,red_blood_cell_count_last,red_blood_cell_count_ascites,red_blood_cell_count_ascites_last,red_blood_cell_count_csf,red_blood_cell_count_csf_last,red_blood_cell_count_pleural,red_blood_cell_count_pleural_last,red_blood_cell_count_urine,red_blood_cell_count_urine_last,respiratory_rate,respiratory_rate_last,respiratory_rate_set,respiratory_rate_set_last,sodium,sodium_last,systemic_vascular_resistance,systemic_vascular_resistance_last,systolic_blood_pressure,systolic_blood_pressure_last,temperature,temperature_last,tidal_volume_observed,tidal_volume_observed_last,tidal_v

In [44]:
temp_patient_data = patient_data.copy()

In [45]:
#Add static variable to dataset 
dummy_sex = pd.get_dummies(patients['gender'])
dummy_insurance = pd.get_dummies(patients['insurance'])
dummy_insurance.columns = [c.replace(' ', '_') for c in dummy_insurance.columns]
dummy_insurance

,,,Government,Medicaid,Medicare,Private,Self_Pay
subject_id,hadm_id,icustay_id,,,,,
3,145834,211552,0,0,1,0,0
4,185777,294638,0,0,0,1,0
6,107064,228232,0,0,1,0,0
9,150750,220597,0,1,0,0,0
11,194540,229441,0,0,0,1,0
...,...,...,...,...,...,...,...
99985,176670,279638,0,0,0,1,0
99991,151118,226241,0,0,0,1,0
99992,197084,242052,0,0,1,0,0


In [46]:
dummy_sex.F

subject_id  hadm_id  icustay_id
3           145834   211552        0
4           185777   294638        1
6           107064   228232        1
9           150750   220597        0
11          194540   229441        1
                                  ..
99985       176670   279638        0
99991       151118   226241        0
99992       197084   242052        1
99995       137810   229633        1
99999       113369   246512        1
Name: F, Length: 30495, dtype: uint8

In [47]:
male = pd.Series(dummy_sex.M)
female = pd.Series(dummy_sex.F)
Government_insurance = pd.Series(dummy_insurance.Government)
Medicaid_insurance = pd.Series(dummy_insurance.Medicaid)
Medicare_insurance = pd.Series(dummy_insurance.Medicare)
Private_insurance = pd.Series(dummy_insurance.Private)
Self_Pay_insurance = pd.Series(dummy_insurance.Self_Pay)

In [70]:
male

subject_id  hadm_id  icustay_id
3           145834   211552        1
4           185777   294638        0
6           107064   228232        0
9           150750   220597        1
11          194540   229441        0
                                  ..
99985       176670   279638        1
99991       151118   226241        1
99992       197084   242052        0
99995       137810   229633        0
99999       113369   246512        0
Name: M, Length: 30495, dtype: uint8

In [48]:
# temp_patient_data = pd.concat([temp_patient_data, dummy_sex], axis=1,ignore_index=True)
temp_patient_data['male'] = male.values
temp_patient_data['female'] = female.values
temp_patient_data['Government_insurance'] = Government_insurance.values
temp_patient_data['Medicaid_insurance'] = Medicaid_insurance.values
temp_patient_data['Medicare_insurance'] = Medicare_insurance.values
temp_patient_data['Private_insurance'] = Private_insurance.values
temp_patient_data['Self_Pay_insurance'] = Self_Pay_insurance.values
temp_patient_data


,alanine_aminotransferase,alanine_aminotransferase_last,albumin,albumin_last,albumin_ascites,albumin_ascites_last,albumin_pleural,albumin_pleural_last,albumin_urine,albumin_urine_last,alkaline_phosphate,alkaline_phosphate_last,anion_gap,anion_gap_last,asparate_aminotransferase,asparate_aminotransferase_last,basophils,basophils_last,bicarbonate,bicarbonate_last,bilirubin,bilirubin_last,blood_urea_nitrogen,blood_urea_nitrogen_last,calcium,calcium_last,calcium_ionized,calcium_ionized_last,calcium_urine,calcium_urine_last,cardiac_index,cardiac_index_last,cardiac_output_fick,cardiac_output_fick_last,cardiac_output_thermodilution,cardiac_output_thermodilution_last,central_venous_pressure,central_venous_pressure_last,chloride,chloride_last,chloride_urine,chloride_urine_last,cholesterol,cholesterol_last,cholesterol_hdl,cholesterol_hdl_last,cholesterol_ldl,cholesterol_ldl_last,co2,co2_last,"co2_(etco2,_pco2,_etc.)","co2_(etco2,_pco2,_etc.)_last",creatinine,creatinine_last,creatinine_ascites,creatinine_ascites_last,creatinine_body_fluid,creatinine_body_fluid_last,creatinine_pleural,creatinine_pleural_last,creatinine_urine,creatinine_urine_last,diastolic_blood_pressure,diastolic_blood_pressure_last,eosinophils,eosinophils_last,fibrinogen,fibrinogen_last,fraction_inspired_oxygen,fraction_inspired_oxygen_last,fraction_inspired_oxygen_set,fraction_inspired_oxygen_set_last,glascow_coma_scale_total,glascow_coma_scale_total_last,glucose,glucose_last,heart_rate,heart_rate_last,height,height_last,hematocrit,hematocrit_last,hemoglobin,hemoglobin_last,lactate,lactate_last,lactate_dehydrogenase,lactate_dehydrogenase_last,lactate_dehydrogenase_pleural,lactate_dehydrogenase_pleural_last,lactic_acid,lactic_acid_last,lymphocytes,lymphocytes_last,lymphocytes_ascites,lymphocytes_ascites_last,lymphocytes_atypical,lymphocytes_atypical_last,lymphocytes_atypical_csl,lymphocytes_atypical_csl_last,lymphocytes_body_fluid,lymphocytes_body_fluid_last,lymphocytes_percent,lymphocytes_percent_last,lymphocytes_pleural,lymphocytes_pleural_last,magnesium,magnesium_last,mean_blood_pressure,mean_blood_pressure_last,mean_corpuscular_hemoglobin,mean_corpuscular_hemoglobin_last,mean_corpuscular_hemoglobin_concentration,mean_corpuscular_hemoglobin_concentration_last,mean_corpuscular_volume,mean_corpuscular_volume_last,monocytes,monocytes_last,monocytes_csl,monocytes_csl_last,neutrophils,neutrophils_last,oxygen_saturation,oxygen_saturation_last,partial_pressure_of_carbon_dioxide,partial_pressure_of_carbon_dioxide_last,partial_pressure_of_oxygen,partial_pressure_of_oxygen_last,partial_thromboplastin_time,partial_thromboplastin_time_last,peak_inspiratory_pressure,peak_inspiratory_pressure_last,ph,ph_last,ph_urine,ph_urine_last,phosphate,phosphate_last,phosphorous,phosphorous_last,plateau_pressure,plateau_pressure_last,platelets,platelets_last,positive_end-expiratory_pressure,positive_end-expiratory_pressure_last,positive_end-expiratory_pressure_set,positive_end-expiratory_pressure_set_last,post_void_residual,post_void_residual_last,potassium,potassium_last,potassium_serum,potassium_serum_last,prothrombin_time_inr,prothrombin_time_inr_last,prothrombin_time_pt,prothrombin_time_pt_last,pulmonary_artery_pressure_mean,pulmonary_artery_pressure_mean_last,pulmonary_artery_pressure_systolic,pulmonary_artery_pressure_systolic_last,pulmonary_capillary_wedge_pressure,pulmonary_capillary_wedge_pressure_last,red_blood_cell_count,red_blood_cell_count_last,red_blood_cell_count_ascites,red_blood_cell_count_ascites_last,red_blood_cell_count_csf,red_blood_cell_count_csf_last,red_blood_cell_count_pleural,red_blood_cell_count_pleural_last,red_blood_cell_count_urine,red_blood_cell_count_urine_last,respiratory_rate,respiratory_rate_last,respiratory_rate_set,respiratory_rate_set_last,sodium,sodium_last,systemic_vascular_resistance,systemic_vascular_resistance_last,systolic_blood_pressure,systolic_blood_pressure_last,temperature,temperature_last,tidal_volume_observed,tidal_volume_observed_last,tidal_v

In [49]:
temp_patient_data.to_csv('first_vital_patient_data.csv',index=False)
# total_los_patient.to
total_los = patients.total_los
type(total_los)
total_los.to_csv('total_los_patients.csv')

In [37]:
np.array(total_los)

array([10.7847,  7.759 , 16.3646, ...,  2.9951,  3.2188,  5.3854])

In [ ]:
# Adding static feature of patients to patient_data

# gender = pd.Series(list(patients.gender))
# ethnicity = pd.Series(list(patients.ethnicity))
# age = pd.Series(list(patients.age))
# insurance = pd.Series(list(patients.insurance))

# patient_data['gender'] = gender.values
# patient_data['ethnicity'] = ethnicity.values
# patient_data['age'] = age.values
# patient_data['insurance'] = insurance.values

In [54]:
working_patient_data = patient_data.copy()

In [58]:
# working_patient_data.columns[]
patients.shape[0]

30495

In [59]:
drop_list = []

for i in range(0,working_patient_data.shape[1]):
    if working_patient_data.iloc[:,i].isnull().sum() / working_patient_data.shape[0] > 0.99:
        drop_list.append(working_patient_data.columns[i])
        
working_patient_data.drop(drop_list,axis=1,inplace=True)

# working_patient_data.drop(['red blood cell count ascites'],axis=1,inplace=True)
# working_patient_data.drop(['red blood cell count csf'],axis=1,inplace=True)
# working_patient_data.drop(['red blood cell count pleural'],axis=1,inplace=True)
# working_patient_data.drop(['red blood cell count urine'],axis=1,inplace=True)
# working_patient_data.drop(['red blood cell count'],axis=1,inplace=True)
# working_patient_data.drop(['alanine aminotransferase'],axis=1,inplace=True)


# working_patient_data = working_patient_data.fillna(working_patient_data.mean())

In [60]:
working_patient_data.shape

(30495, 95)

In [61]:
from sklearn.impute import SimpleImputer
missingvalues = SimpleImputer(missing_values = np.nan, strategy = 'median')
missingvalues = missingvalues.fit(working_patient_data)
working_patient_data = missingvalues.transform(working_patient_data)

In [41]:
# list(working_patient_data[:,83])

In [32]:
# get y value
total_los = np.array(patients.total_los)
sum(np.isnan(total_los))
total_los.to_csv()

NameError: name 'df' is not defined

In [67]:
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# mm_scaler = MinMaxScaler()
# working_patient_data = mm_scaler.fit_transform(working_patient_data)

X_train, X_test, y_train, y_test = train_test_split(working_patient_data, total_los, test_size=0.3, random_state=1)

In [71]:
np.random.seed(1)

In [39]:
conda install -c conda-forge xgboost


Solving environment: done

## Package Plan ##

  environment location: /home/student/miniconda3/envs/huy_thesis

  added / updated specs:
    - xgboost


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py37hc8dfbb8_0         151 KB  conda-forge
    libxgboost-1.0.2           |       he1b5a44_1         2.8 MB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    py-xgboost-1.0.2           |   py37hc8dfbb8_1         2.2 MB  conda-forge
    xgboost-1.0.2              |   py37h3340039_1          11 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         7.5 MB

The following NEW package

In [ ]:
clf = SVR(kernel='linear')
clf.fit(X_train, y_train)

training_loss = mean_absolute_error(y_train, clf.predict(X_train))
testing_loss = mean_absolute_error(y_test, clf.predict(X_test))

print('Training loss is {}'.format(training_loss))
print('Testing loss is {}'.format(testing_loss))